In [240]:
#import pandas, matplotlib and plotly libaries
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px

#read data from csv file into vehicles_df dataframe. No delimiters are specified, default comma is used.
vehicles_df = pd.read_csv('../vehicles_us.csv')

#display vehicles_df information
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


Looks like there may be some missing and possibly duplicate values in the dataframe, based on the call to the info() method. We'll investigate closer:

In [241]:

#Check for duplicated rows in the dataframe
print('There are',vehicles_df.duplicated().sum(),'duplicated rows in the dataframe.')
print('')

#define a function named check_missing to check for missing values. Includes a 
#for loop which iterates through each column in the vehicles_df dataframe and checks for columns where
#there are missing values and prints the total amount of missing values for each column
def check_missing():
    none_missing = 0
    for column in vehicles_df:
        missing = vehicles_df[column].isna().sum()
        if missing > 0:
            print("Column", column, "has", missing, "missing values.")
        else:
            none_missing += 1

    print('')
    print('There are', none_missing, 'columns with no missing values.')
    print('')

#call check_missing function
check_missing()

There are 0 duplicated rows in the dataframe.

Column model_year has 3619 missing values.
Column cylinders has 5260 missing values.
Column odometer has 7892 missing values.
Column paint_color has 9267 missing values.
Column is_4wd has 25953 missing values.

There are 8 columns with no missing values.



We've found some issues with the dataframe columns and the data contained within. In the following code block we will be making some changes to the dataframe:

In [242]:

#Fills missing values in model_year column with value of 0 and converts entire column to integer type 
vehicles_df['model_year'] = vehicles_df['model_year'].fillna('0').astype('int')

#Fills missing values in cylinders column with value of 0 and converts entire column to integer type
vehicles_df['cylinders'] = vehicles_df['cylinders'].fillna('0').astype('int')

#Fills missing values in odometer column with value of 0 and converts entire column to integer type
vehicles_df['odometer'] = vehicles_df['odometer'].fillna('0').astype('int')

#Fills missing values in paint_color column with string "unspecified" and converts entire column to string (object Dtype)
vehicles_df['paint_color'] = vehicles_df['paint_color'].fillna('unspecified').astype('str')

#Fills missing values in is_4wd column with string '2wd' (two wheel drive) and converts entire column to string (object Dtype)
vehicles_df['is_4wd'] = vehicles_df['is_4wd'].fillna('2wd').astype('str')

#Replaces values of 1 (with the assumption that a value of 1 means that vehicle is 4wd) with string '4wd'
vehicles_df['is_4wd'] = vehicles_df['is_4wd'].replace(['1.0'], '4wd')

#Renames is_4wd column to something more intuitive - "drivetrain"
vehicles_df = vehicles_df.rename(columns={'is_4wd': 'drivetrain'})

After modifying the dataframe, we then call the check_missing function again to show that we have successfully modified the dataframe:

In [243]:
#call check_missing function to see that the changes we made are successful. Since there are 13 columns, we expect to see 13 columns
#with no missing values as the result.
check_missing()


There are 13 columns with no missing values.



In the following segment we will take a closer look at the values in the dataset. We want to be sure they make sense, and any extreme outliers will be removed.

In [244]:
#check to see the vehicle prices make sense
print(vehicles_df['price'].max())

#Display the entries with values greater than 90000
display(vehicles_df.loc[vehicles_df['price'] > 90000])

#Remove the illogical entries from the vehicles_df dataframe
vehicles_df.drop(vehicles_df.loc[vehicles_df['price'] > 90000].index, inplace=True)

#resets the index values after removing the nonsensical entries
vehicles_df = vehicles_df.reset_index(drop=True)

#display info for vehicle_df to show the removed entries
display(vehicles_df.info())



375000


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,drivetrain,date_posted,days_listed
1309,189000,2014,ford f-150,good,6,gas,151248,automatic,truck,black,2wd,2019-03-02,56
1668,189000,2014,ford f-150,good,6,gas,151248,automatic,truck,unspecified,2wd,2019-03-20,21
4113,90577,2018,ford f-150,excellent,6,gas,6448,automatic,truck,black,4wd,2018-07-29,19
11359,300000,2015,ram 2500,excellent,0,diesel,0,automatic,truck,grey,4wd,2018-10-15,39
12504,375000,1999,nissan frontier,good,6,gas,115000,automatic,pickup,blue,4wd,2018-05-19,21
20700,109999,0,chevrolet corvette,new,8,gas,35,automatic,coupe,white,2wd,2018-09-07,74
22833,115000,2012,ford f-250 super duty,good,8,diesel,225000,automatic,truck,white,2wd,2018-12-08,53
24999,140000,2004,ram 2500,good,6,diesel,249000,automatic,truck,red,4wd,2019-01-22,8
27375,189000,2014,ford f-150,good,6,gas,151248,automatic,truck,black,2wd,2018-09-25,72
29810,123456,2000,chevrolet suburban,good,8,gas,123456,manual,truck,silver,2wd,2018-05-09,40


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51506 entries, 0 to 51505
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   price         51506 non-null  int64 
 1   model_year    51506 non-null  int32 
 2   model         51506 non-null  object
 3   condition     51506 non-null  object
 4   cylinders     51506 non-null  int32 
 5   fuel          51506 non-null  object
 6   odometer      51506 non-null  int32 
 7   transmission  51506 non-null  object
 8   type          51506 non-null  object
 9   paint_color   51506 non-null  object
 10  drivetrain    51506 non-null  object
 11  date_posted   51506 non-null  object
 12  days_listed   51506 non-null  int64 
dtypes: int32(3), int64(2), object(8)
memory usage: 4.5+ MB


None

We removed several entries with prices far exceeding that of normal market value for the given makes and models. Next we will look at the price versus the condition of the vehicle.

In [270]:
#fig = px.histogram(vehicles_df, x="condition", y="price") 
#fig = px.scatter(vehicles_df, x="condition", y="price") 

#create df with average price vs condition of vehicles data
pc_df = vehicles_df.groupby('condition')['price'].mean().round(0).astype('int')

#plotly chart settings
fig = px.bar(pc_df, y="price",
title="Average Price of Vehicle listings versus their Condition",
labels={"price": "Average Price ($)", "condition": "Condition"}) 
fig.update_xaxes(categoryorder='array', categoryarray= ['new', 'like new', 'excellent', 'good', 'fair', 'salvage'])
# showing the plot
fig.show()